In [1]:
# Basics and Plotting
import pandas as pd
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
from itertools import chain, combinations
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Sklearn Models
import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score, cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

# Alternative models
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf

<ipython-input-1-645cd4fdeda9>:13: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/jprichmond20/2020ElectionStudy/main/anes_timeseries_2020_csv_20220210.csv")

C:\Users\JP\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = df[["V201004", "V201005", "V201008", 
                    "V201024", "V202110x", "V201030", "V201651", 
                    "V201645", "V201628", "V201627", "V201626", 
                    "V201622", "V201620", "V201607", "V201606", 
                    "V201602", "V201600", "V201601", "V201594", 
                    "V201589", "V201575", "V201565x", "V201550", 
                    "V201549x", "V201540", "V201531", 
                    "V201530y", "V201515", "V201511x", "V201508", 
                    "V201507x", "V201462", "V201453", "V201435", 
                    "V201433", "V201432x", "V201426x", "V201417", 
                    "V201416", "V201411x", "V201393", "V201379", 
                    "V201377", "V201366", "V201364", "V201359x", 
                    "V201335", "V201244", "V201242", "V201241", 
                    "V201240", "V201239", "V201237", "V201231x", 
                    "V201232", "V201220", "V201216", "V201156", 
                    "V201157", "V201122", "V201123", "V201121", 
                    "V201114", "V201100", "V201103", "V201036", 
                   "V201018"]]

In [4]:
df = df.iloc[:,[4,64,63,49,2,17,56,38,42,28,50,15,9,11,6,58,65,8,27]]

In [5]:
df = df.rename(columns = {'V202110x': 'VFor','V201103': 'VFor2016' , 'V201100': 'Likely2V', 'V201241':'PBetterImmigration', 'V201008':'R2V', 'V201601':'SexualOrientation', 'V201216':'CareWhoWins', 'V201416':'GayMarriage', 'V201377':'TrustMedia', 'V201511x':'LevelOfEducation', 'V201240':'PBetterHealthcare', 'V201602':'JustifiedPViolence', 'V201627':'SelfCensor', 'V201622':'ConcernedPay4Healthcare', 'V201651':'SatisfiedWLife', 'V201157':'FThermometerR', 'V201036':'CandidatePref', 'V201628':'GunsOwned', 'V201515':'DiplomaOrGED'})

In [6]:
df.head()

,VFor,VFor2016,Likely2V,PBetterImmigration,R2V,SexualOrientation,CareWhoWins,GayMarriage,TrustMedia,LevelOfEducation,PBetterHealthcare,JustifiedPViolence,SelfCensor,ConcernedPay4Healthcare,SatisfiedWLife,FThermometerR,CandidatePref,GunsOwned,DiplomaOrGED
0,-1,2,1,5,1,1,1,3,1,4,5,1,1,1,1,85,-1,3,-1
1,3,5,2,3,1,1,1,1,1,3,3,1,2,5,3,50,-1,0,2
2,1,1,1,1,1,1,1,1,3,2,1,3,3,2,3,0,-1,0,-1
3,1,1,2,3,1,1,2,1,4,3,3,1,4,4,1,70,-1,0,-1
4,2,2,1,5,1,1,1,2,2,5,4,1,4,1,2,70,-1,0,-1


In [7]:
df['VFor'].value_counts()

 1    3537
 2    2582
-1    1920
-9      81
 3      74
 5      60
 4      24
-8       2
Name: VFor, dtype: int64

In [8]:
df2 = df.copy()
f1 = df2['VFor']==1
f2 = df2['VFor']==2
df2['VFor'].where(f1|f2, inplace = True)
df2.head()

,VFor,VFor2016,Likely2V,PBetterImmigration,R2V,SexualOrientation,CareWhoWins,GayMarriage,TrustMedia,LevelOfEducation,PBetterHealthcare,JustifiedPViolence,SelfCensor,ConcernedPay4Healthcare,SatisfiedWLife,FThermometerR,CandidatePref,GunsOwned,DiplomaOrGED
0,NaN,2,1,5,1,1,1,3,1,4,5,1,1,1,1,85,-1,3,-1
1,NaN,5,2,3,1,1,1,1,1,3,3,1,2,5,3,50,-1,0,2
2,1.0,1,1,1,1,1,1,1,3,2,1,3,3,2,3,0,-1,0,-1
3,1.0,1,2,3,1,1,2,1,4,3,3,1,4,4,1,70,-1,0,-1
4,2.0,2,1,5,1,1,1,2,2,5,4,1,4,1,2,70,-1,0,-1


In [9]:
df2 = df2.dropna()
df2.head()

,VFor,VFor2016,Likely2V,PBetterImmigration,R2V,SexualOrientation,CareWhoWins,GayMarriage,TrustMedia,LevelOfEducation,PBetterHealthcare,JustifiedPViolence,SelfCensor,ConcernedPay4Healthcare,SatisfiedWLife,FThermometerR,CandidatePref,GunsOwned,DiplomaOrGED
2,1.0,1,1,1,1,1,1,1,3,2,1,3,3,2,3,0,-1,0,-1
3,1.0,1,2,3,1,1,2,1,4,3,3,1,4,4,1,70,-1,0,-1
4,2.0,2,1,5,1,1,1,2,2,5,4,1,4,1,2,70,-1,0,-1
5,1.0,-1,1,2,1,1,1,1,4,3,2,1,3,2,4,15,-1,0,-1
6,2.0,5,1,3,2,1,3,1,2,3,3,2,5,3,1,75,-1,0,-1


In [10]:
df2.head()

,VFor,VFor2016,Likely2V,PBetterImmigration,R2V,SexualOrientation,CareWhoWins,GayMarriage,TrustMedia,LevelOfEducation,PBetterHealthcare,JustifiedPViolence,SelfCensor,ConcernedPay4Healthcare,SatisfiedWLife,FThermometerR,CandidatePref,GunsOwned,DiplomaOrGED
2,1.0,1,1,1,1,1,1,1,3,2,1,3,3,2,3,0,-1,0,-1
3,1.0,1,2,3,1,1,2,1,4,3,3,1,4,4,1,70,-1,0,-1
4,2.0,2,1,5,1,1,1,2,2,5,4,1,4,1,2,70,-1,0,-1
5,1.0,-1,1,2,1,1,1,1,4,3,2,1,3,2,4,15,-1,0,-1
6,2.0,5,1,3,2,1,3,1,2,3,3,2,5,3,1,75,-1,0,-1


In [16]:
#1 = Biden 2 = Trump
df2.replace(1.0, "Biden", inplace = True)
df2.replace(2.0, "Trump", inplace = True)

In [17]:
fig1 = px.bar(df2, x=df2['VFor'].unique(), \
              y=df2.groupby('VFor')['VFor'].agg('count'), \
              labels={"x":"Candidate", "y":"# Of Voters"}, \
              color_continuous_scale=px.colors.sequential.Hot, \
              title="Votes For Candidate")
fig1.update_xaxes(type='category')
fig1.update_layout(title=dict(x=0.5),\
                  margin=dict(l=20,r=20,t=60,b=20),\
                  paper_bgcolor="#D3D3D3")

In [18]:
df.head()

,VFor,VFor2016,Likely2V,PBetterImmigration,R2V,SexualOrientation,CareWhoWins,GayMarriage,TrustMedia,LevelOfEducation,PBetterHealthcare,JustifiedPViolence,SelfCensor,ConcernedPay4Healthcare,SatisfiedWLife,FThermometerR,CandidatePref,GunsOwned,DiplomaOrGED
0,-1,2,1,5,1,1,1,3,1,4,5,1,1,1,1,85,-1,3,-1
1,3,5,2,3,1,1,1,1,1,3,3,1,2,5,3,50,-1,0,2
2,1,1,1,1,1,1,1,1,3,2,1,3,3,2,3,0,-1,0,-1
3,1,1,2,3,1,1,2,1,4,3,3,1,4,4,1,70,-1,0,-1
4,2,2,1,5,1,1,1,2,2,5,4,1,4,1,2,70,-1,0,-1


In [20]:
df3 = df.copy()
df3.head()

,VFor,VFor2016,Likely2V,PBetterImmigration,R2V,SexualOrientation,CareWhoWins,GayMarriage,TrustMedia,LevelOfEducation,PBetterHealthcare,JustifiedPViolence,SelfCensor,ConcernedPay4Healthcare,SatisfiedWLife,FThermometerR,CandidatePref,GunsOwned,DiplomaOrGED
0,-1,2,1,5,1,1,1,3,1,4,5,1,1,1,1,85,-1,3,-1
1,3,5,2,3,1,1,1,1,1,3,3,1,2,5,3,50,-1,0,2
2,1,1,1,1,1,1,1,1,3,2,1,3,3,2,3,0,-1,0,-1
3,1,1,2,3,1,1,2,1,4,3,3,1,4,4,1,70,-1,0,-1
4,2,2,1,5,1,1,1,2,2,5,4,1,4,1,2,70,-1,0,-1


In [ ]:
fig2 = px.bar(df3, x=df3['VFor'].unique(), \
              y=df3.groupby('VFor')['VFor'].agg('count'), \
              labels={"x":"Candidate", "y":"# Of Voters"}, \
              color_continuous_scale=px.colors.sequential.Hot, \
              title="Votes For Candidate")
fig2.update_xaxes(type='category')
fig2.update_layout(title=dict(x=0.5),\
                  margin=dict(l=20,r=20,t=60,b=20),\
                  paper_bgcolor="#D3D3D3")

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.config.suppress_callback_exceptions = True

In [ ]:
app.layout = html.Div(children=[\
    html.Div(children=[
        html.H1(children="Baseball Statistics", style={'textAlign':'center'},className="header-text"),
        html.H2(children="JP Richmond and Alex Dufour",style={'fontSize':"30px",'textAlign':'center'},className="names-header"),\
]),
    html.Div(\
        children=[html.Div(children='yearID',\
                                style={'fontSize':"20px"},\
                                className='menu-title-bar'),\
                                dcc.Dropdown(id='year-selection',\
                                options=[{'label':yearID,'value':yearID}\
                                for yearID in baseball2.yearID.unique()],\
                                value='2016',\
                                clearable=False,\
                                searchable=False,\
                                className='dropdown',\
                                style={'fontSize':"20px",'textAlign':'center'})],\
             className='menu'),
    html.Div(children=[\
        html.Div(\
            children = dcc.Graph(\
                    id = 'barHR',\
                    figure = fig2,\
                  #  config={"displayModeBar": False},
                ),\
                style={'width': '50%', 'display': 'inline-block'},\
            ),\
                html.Div(\
                children = dcc.Graph(\
                    id = 'barHRLg',\
                    figure = fig1,\
                    #config={"displayModeBar": False},
                ),\
                style={'width': '50%', 'display': 'inline-block'},\
            ),\
                html.Div(\
                children = dcc.Graph(\
                    id = 'barSO',\
                    figure = fig3,\
                    #config={"displayModeBar": False},
                ),\
                         style={'width': '100%', 'display': 'inline-block'},),\
])])

In [ ]:
@app.callback(Output("barHR","figure"),Input("years","value"))
def update_charts(years):
    filtered_data = baseball2[baseball2["yearID"] == years]
    barHR = px.bar(filtered_data,x="teamID",y="HR",title="Home Runs by Team",color_continuous_scale=px.colors.sequential.Plasma)
    barHR.update_layout(xaxis_tickangle=20,title=dict(x=0.5),xaxis_tickfont=dict(size=8),yaxis_tickfont=dict(size=8),paper_bgcolor="LightSteelBlue",margin=dict(l=30,r=20,t=50,b=20))
    return barHR
@app.callback(Output("barHRLg","figure"),Input("years","value"))
def update_charts(years):
    filtered_data = baseball2[baseball2["yearID"] == years]
    barHRLg = px.bar(filtered_data,x="lgID",y="HR",title="Home Runs by League",color_continuous_scale=px.colors.sequential.Plasma)
    barHRLg.update_layout(xaxis_tickangle=20,title=dict(x=0.5),xaxis_tickfont=dict(size=8),yaxis_tickfont=dict(size=8),paper_bgcolor="LightSteelBlue",margin=dict(l=30,r=20,t=50,b=20))
    return barHRLg
@app.callback(Output("barSO","figure"),Input("years","value"))
def update_charts(years):
    filtered_data = baseball2[baseball2["yearID"] == years]
    barSO = px.bar(filtered_data,x="teamID",y="SO",title="Strikeouts by Team",color_continuous_scale=px.colors.sequential.Plasma)
    barSO.update_layout(xaxis_tickangle=20,title=dict(x=0.5),xaxis_tickfont=dict(size=8),yaxis_tickfont=dict(size=8),paper_bgcolor="LightSteelBlue",margin=dict(l=30,r=20,t=50,b=20))
    return barSO